In [1]:
import pandas as pd
import numpy as np
import psycopg2

/Users/apple/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Get Commit id and Labels

In [20]:
df_origin =  pd.read_excel('commit data.xlsx', sheet_name= 'Commit Data')
print(df_origin.head())
print(df_origin.shape)

  commit id   label1 label2 label3       label4       label5         label6
0   0b9ea98  Testing    NaN    NaN          NaN          NaN    Feature Add
1   980d9f8      NaN    NaN    NaN          NaN          NaN    Refactoring
2   e9d5060  Testing    NaN    NaN          NaN      Bug fix    Feature Add
3   a63639b  Testing    NaN    NaN  Maintenance        Build    Feature Add
4   a611d64      NaN    NaN    NaN          NaN  Module Move  Module Remove
(925, 7)


### Get Multiple Labels

In [21]:
def create_label(row):
    result = []
    for i in range(1,7):
        if pd.isnull(row['label' + str(i)]) == False:
            result.append(row['label' + str(i)])
    
    return result

df_origin['categories'] = df_origin.apply(lambda row : create_label(row), axis = 1)
df_origin.to_csv('orignal_commit_data.csv')
df_origin.head()

,commit id,label1,label2,label3,label4,label5,label6,categories
0,0b9ea98,Testing,NaN,NaN,NaN,NaN,Feature Add,"[Testing, Feature Add]"
1,980d9f8,NaN,NaN,NaN,NaN,NaN,Refactoring,[Refactoring]
2,e9d5060,Testing,NaN,NaN,NaN,Bug fix,Feature Add,"[Testing, Bug fix, Feature Add]"
3,a63639b,Testing,NaN,NaN,Maintenance,Build,Feature Add,"[Testing, Maintenance, Build, Feature Add]"
4,a611d64,NaN,NaN,NaN,NaN,Module Move,Module Remove,"[Module Move, Module Remove]"


### Make New Feature and Label Dataset

In [28]:
df = df_origin.copy()
df = df.drop(['label1','label2','label3','label4','label5','label6'], axis = 1)
df.head()

,commit id,categories
0,0b9ea98,"[Testing, Feature Add]"
1,980d9f8,[Refactoring]
2,e9d5060,"[Testing, Bug fix, Feature Add]"
3,a63639b,"[Testing, Maintenance, Build, Feature Add]"
4,a611d64,"[Module Move, Module Remove]"


In [29]:
try:
    conn =  psycopg2.connect("dbname='squad' user='apple' password = 'mst123456' host='localhost'")
except:
    print ("I am unable to connect to the database")  

cur = conn.cursor()

csha_list = df['commit id'].values
print(csha_list[0])
print(csha_list[96])

commit_message = []
for index, csha in enumerate(csha_list):
    qr =  "SELECT message FROM commits WHERE csha LIKE '{}%'".format(csha_list[index].strip())
    cur.execute(qr)
    fetched_row = cur.fetchone()
    if fetched_row != None:
        if index < 5:
            print(index, fetched_row[0])
        commit_message.append(fetched_row[0])
    else:
        if index < 5:
            print('None')
        commit_message.append('None')

0b9ea98
9dcf08d
0 [CALCITE-1124] Add TIMESTAMPADD, TIMESTAMPDIFF functions (Arina Ielchiieva)

Close apache/calcite#204

1 Move code from JdbcImplementor and JdbcRules to new class SqlImplementor

2 [CALCITE-968] Stream-to-relation and stream-to-stream joins (Milinda Pathirage)

Rule to transform Delta(Scan(constant-table)) to Empty;
fix NullPointerException in PruneEmptyRules.

3 [CALCITE-912] Add Avatica OpenConnectionRequest (Bruno Dumon)

Goal: passing of connection properties (the 'info') from the remote
avatica driver to the corresponding server-side connection.

Changes include:
 * in Meta: explicit opening of connections: there is a new
   openConnection() call. Client decides on the connection id.
   Implicit creation of unknown connections is gone.
 * JdbcMeta: default connection is gone, all methods have been
   extended with a connection id.
 * correspondingly, the various Service.Request's now carry a
   connection id (will make fixing CALCITE-871 easy)
 * (unrelated, but 

In [30]:
df['commit_message'] = commit_message
df.head()

,commit id,categories,commit_message
0,0b9ea98,"[Testing, Feature Add]","[CALCITE-1124] Add TIMESTAMPADD, TIMESTAMPDIFF..."
1,980d9f8,[Refactoring],Move code from JdbcImplementor and JdbcRules t...
2,e9d5060,"[Testing, Bug fix, Feature Add]",[CALCITE-968] Stream-to-relation and stream-to...
3,a63639b,"[Testing, Maintenance, Build, Feature Add]",[CALCITE-912] Add Avatica OpenConnectionReques...
4,a611d64,"[Module Move, Module Remove]",[CALCITE-296] Re-organize package structure;\n...


### Export to CSV

In [31]:
df.to_csv('commit_data.csv')